<a href="https://colab.research.google.com/github/heidaralavi/Fetch-daily-product/blob/main/compairing%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/heidaralavi/Fetch-daily-product.git

Cloning into 'Fetch-daily-product'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 117 (delta 40), reused 80 (delta 14), pack-reused 0
Receiving objects: 100% (117/117), 22.87 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [2]:
import pandas as pd
import numpy as np

In [3]:
f='/content/Fetch-daily-product/Daily production98-1400.xlsx'
production_df = pd.DataFrame()
xl = pd.ExcelFile(f)
sheet_names=xl.sheet_names
for sheet_name in sheet_names:
    df_temp= pd.read_excel(f, sheet_name=sheet_name )
    production_df= production_df.append(df_temp,ignore_index = True)
        
production_df

,shamsidate,daily_production
0,13980101,11131
1,13980102,11514
2,13980103,11849
3,13980104,11722
4,13980105,9127
...,...,...
1079,14001213,13151
1080,14001214,13069
1081,14001215,13089
1082,14001216,13148


In [4]:
temp_arr=[]
for item in production_df.iloc[:,0].values:
  temp=(str(item))
  temp = temp[:4]+'/'+temp[4:6]+'/'+temp[6:]
  temp_arr.append(temp)

production_df.insert(0,'jdate' ,temp_arr)
production_df.drop(columns='shamsidate',inplace=True)
production_df

,jdate,daily_production
0,1398/01/01,11131
1,1398/01/02,11514
2,1398/01/03,11849
3,1398/01/04,11722
4,1398/01/05,9127
...,...,...
1079,1400/12/13,13151
1080,1400/12/14,13069
1081,1400/12/15,13089
1082,1400/12/16,13148


In [5]:
final_df= pd.read_excel('/content/Fetch-daily-product/final_data.xlsx',index_col=0)

In [6]:
final_df

,date,jdate,M+L+E(ton/day),H(ton/day),A(ton/day),G(ton/day)
0,2019-03-21,1398/01/01,11131,860.78,12131.0,551.00
1,2019-03-22,1398/01/02,11514,789.73,12385.3,543.00
2,2019-03-23,1398/01/03,11849,854.57,12415.0,530.00
3,2019-03-24,1398/01/04,11722,862.67,12278.4,524.00
4,2019-03-25,1398/01/05,9127,585.07,9948.0,385.00
...,...,...,...,...,...,...
1079,2022-03-04,1400/12/13,13151,819.96,14243.0,164.00
1080,2022-03-05,1400/12/14,13069,755.44,13928.2,156.00
1081,2022-03-06,1400/12/15,13089,265.12,13950.7,162.00
1082,2022-03-07,1400/12/16,13148,480.42,13935.3,154.00


In [7]:
merged_df = pd.merge(final_df[['jdate','M+L+E(ton/day)']], production_df[['jdate','daily_production']],
                     on=['jdate'], how='outer')

In [8]:
merged_df

,jdate,M+L+E(ton/day),daily_production
0,1398/01/01,11131,11131
1,1398/01/02,11514,11514
2,1398/01/03,11849,11849
3,1398/01/04,11722,11722
4,1398/01/05,9127,9127
...,...,...,...
1079,1400/12/13,13151,13151
1080,1400/12/14,13069,13069
1081,1400/12/15,13089,13089
1082,1400/12/16,13148,13148


In [9]:
merged_df.insert(3,'difference',merged_df['M+L+E(ton/day)']-merged_df['daily_production'])

In [10]:
merged_df[merged_df['difference'] != 0]

,jdate,M+L+E(ton/day),daily_production,difference
92,1398/03/31,6193,14373,-8180
185,1398/06/31,0,10847,-10847
275,1398/09/30,11382,11204,178
364,1398/12/29,11955,12021,-66
550,1399/06/31,9510,9610,-100
709,1399/12/09,11211,11111,100
715,1399/12/15,11447,15486,-4039
832,1400/04/09,11152,11088,64
896,1400/06/11,12772,12662,110
1006,1400/09/30,13277,13270,7
